In [47]:
import pickle
from tqdm import tqdm
import pandas as pd
import json


import os 

os.chdir('c:/Users/Domen/IronHack/01_projects/IH_final_project_data-jobs')
print(os.getcwd())


c:\Users\Domen\IronHack\01_projects\IH_final_project_data-jobs


In [48]:

from src.myModules import my_NLP as myNLP

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
print(os.getcwd())
with open("config/config.json", 'r') as f:
    config = json.load(f)
    
NLP_data_path = config['NLP_data_path']

print(NLP_data_path)


nlp_df_input = 'NLP_title'
nlp_df_output_name = 'skills_df'

c:\Users\Domen\IronHack\01_projects\IH_final_project_data-jobs
data/NLP_data/


## Test Code - NLP

load cleaned dataframe

In [50]:
with open(f"{NLP_data_path}{nlp_df_input}.pkl", "rb") as file:
    df = pickle.load(file)


In [61]:
df.fillna("", inplace=True)
df['count_values'] = df['new_job_title'].apply(len)
original_column_with_counts = df[['new_job_title', 'count_values','id', 'title']]

display(original_column_with_counts)

original_column_with_counts["count_values"]

sorted_df = original_column_with_counts.sort_values(by='count_values')
display(sorted_df)


sorted_df = sorted_df.sort_values(by='count_values')

# Get the last 20 entries
last_20_entries = sorted_df.tail(20)

# Print the 'title' values for the last 20 entries
titles_to_print = last_20_entries['title'].tolist()

for title in titles_to_print:
    print(title)

,new_job_title,count_values,id,title
0,[software_engineer],1,3685669741,Software Engineer Fullstack (m/w/d)
1,"[data_analyst, bi_analyst]",2,3719620327,Data Analyst - Business Intelligence
2,[data_engineer],1,3720365801,(Junior) Data Engineer (m/f/d)
3,[data_analyst],1,3672508925,Financial Data Analyst (m/f/d)
4,[bi_analyst],1,3695466781,Business Intelligence Analyst (f/m/d)
...,...,...,...,...
29297,,0,3728033397,Softwareingenieur:in
29298,,0,3671825428,Junior Teamlead Delivery Risk Reporting (m/w/d...
29299,[data_analyst],1,3719993773,Big Data Analystin / Analyst (w/m/d)
29300,[software_developer],1,3699632487,(Junior) Developer (all genders)


,new_job_title,count_values,id,title
14650,,0,3724919518,CONTAINER PLATFORM ENGINEER(ALL GENDERS)
16468,,0,3713484266,Fachinformatiker (w/m/d) für Systemintegration
16465,,0,3713703765,Principal Engineer (m/f/x) - Foreign Exchange ...
16464,,0,3715320605,Mitarbeiter/in im technischen Kundensupport (w...
16463,,0,3706058144,Business Expert Data & Process Management (m/f/x)
...,...,...,...,...
4306,"[ai_engineer, big_data_engineer/specialist, ja...",4,3715658593,Senior AI Architect - Python / Big Data / Java...
5945,"[cloud_engineer, software_developer, it_system...",5,3725476793,Cloud-Developer (m/w/d) mit Managementerfahrun...
21214,"[cloud_engineer, software_developer, it_system...",5,3727809960,Cloud-Developer (m/w/d) mit Managementerfahrun...
19923,"[it_systemadmin, sap_specialist, sap_specialis...",5,3713095897,"Business Unit Lead SAP BI - SAP BW on HANA, BW..."


Technical Lead Developer | Cloud Service Platform | Java, Spring, Cloud | Up to €105k
IT System Engineer / IT Systemadministrator (*)
Senior AI Architect - Python / Big Data / Java / Backend / Home Office (w/m/d)
IT Service Manager SAP Basis & SAP Cloud (m/w/d) – Wiesbaden, Competitive Benefits Package + Chance to join one of Germany’s Leading Brands
Fullstack Software Engineer - Cloud / DevOps / Git / Home Office (m/w/d)
Fullstack Software Engineer - Cloud / DevOps / Git / Home Office (m/w/d)
Software Engineer / IT Consultant - MSc/PhD Graduates / Python / Java  / JavaScript / C/C++ / C# / PHP / SCRUM
Business Intelligence Developer mit Schwerpunkt ETL / Data Warehouse (m/w/d)
IT Manager – IT Product Manager / IT Engineering / Data Analytics (m/f/d)
SAP S/4HANA M&A IT SAP Cloud Architect (m/f/x) in Frankfurt (Main)
Consultant SAP Analytics - SAP Data Warehouse / Business Intelligence (m/w/d)
Fullstack Software Engineer - Cloud / DevOps / Git / Home Office (m/w/d)
Fullstack Developer -

: 

In [ ]:
df.head(2)

In [ ]:
bert_df = df[df['first_match']==True]

In [ ]:
print(df.columns)
nlp_df = df[df['first_match']==True]

# nlp_df = nlp_df[[ 'id', 'company', 'title','new_job_title', 'job_description',  'language', 'url']]

In [ ]:
keywords = ['data_analyst', 'analytics_engineer','bi_analyst', 'ml_engineer', 'business_analyst', 'business_intelligence_analyst', 'data_scientist']

In [ ]:
nlp_df = nlp_df[nlp_df['new_job_title'].apply(lambda job_title: any(skill in keywords for skill in job_title))]

In [ ]:
nlp_df.fillna("", inplace=True)

#### Language detection

In [ ]:
tricky_re_skills = ['r', 'scala', 'sas', 'gcp']

In [ ]:
skills = ['python', 'sql', 'gitlab',' r ','bi-bool','r shiny', 'power bi', 'visual analytics' 'cognos bi','aws', 'spark', 'azure', 'tableau', 'java', 'excel', 'hadoop', 'scala', 
           'snowflake', 'kafka', 'nosql', 'databricks', ' git ', 'redshift', 'airflow', 'oracle', 'sap', 'ad-hoc', 
           'sql server', 'docker', 'kubernetes', 'power_bi', 'c++', 'numpy', 'dplyr', 'matplotlib', 
           'seaborn', 'ggplot', 'hypothesis testing', 'regression analysis', 'predictive analysis', 'bayesian statistics', 
           'scikit', 'tensorflow', 'pytorch', 'xgboost', 'deep learning', 'hadoop', 'bigquery', 'airflow', 'nltk', 'spacy', 
           'nlp', 'aws', 'azure', 'looker', 'ms-office', ' ml ', ' nlp ', ' pandas', 'e2e', ' sas ', ' gcp ']

First round of skill extraction from text

In [ ]:
tqdm.pandas(desc="Processing")
nlp_df['nlp_job_description'] = nlp_df['job_description'].progress_apply(lambda x: x.lower().strip()) # lower job description text
nlp_df = nlp_df.progress_apply(lambda row: myNLP.check_skills(row, skills), axis=1) # use check_skill function to extract skills from text which matches words in skills list


In [ ]:
display(nlp_df.loc[nlp_df["city"]==38])

In [ ]:
nlp_df.columns

In [ ]:
nlp_df = myNLP.prepare_dataset(nlp_df)

In [ ]:
# with open(f"{NLP_data_path}{nlp_df_output_name}.pkl", "wb") as file:
#     pickle.dump(nlp_df, file)

In [ ]:
with open(f"{NLP_data_path}{nlp_df_output_name}.csv", "wb") as file:
    nlp_df.to_csv(file, index=False)

Investigate german and english text to identify further skills to extract 

In [ ]:
filtered_view = nlp_df[nlp_df['new_job_title'].apply(lambda job_title: any(skill in ['data_analyst'] for skill in job_title))] # filter dataset on specific job title
filtered_view.head(20)

In [ ]:
df_ger = nlp_df[nlp_df['language'] == 'de']
df_eng = nlp_df[nlp_df['language'] == 'en']

In [ ]:
df_ger.head(25)

In [ ]:
# df_ger = df_ger.progress_apply(lambda row: myNLP.check_german_style(row), axis=1)
# df_ger.head(2)

In [ ]:
df_ger['nlp_job_description'] = df_ger['nlp_job_description'].progress_apply(myNLP.clean_up)
df_ger['nlp_job_description'] = df_ger['nlp_job_description'].progress_apply(myNLP.stem_and_lemmatize_german) # lemmatize strings
df_ger['nlp_job_description'] = df_ger['nlp_job_description'].progress_apply(myNLP.tokenize) # tokenize
df_ger['nlp_job_description'] = df_ger['nlp_job_description'].progress_apply(myNLP.remove_stopwords_german) #remove stopwords from
df_ger['nlp_job_description'] = df_ger['nlp_job_description'].progress_apply(myNLP.re_blob) # reblob
df_ger['nlp_job_description'] = df_ger['nlp_job_description'].progress_apply(lambda x: x.lower().strip())


In [ ]:
df_ger['nlp_job_description'].head(10)

In [ ]:
# !pip install wordcloud

In [ ]:
myNLP.word_cloud(df_ger, 'nlp_job_description')

english dataset

In [ ]:
df_eng['nlp_job_description'] = df_eng['nlp_job_description'].progress_apply(myNLP.clean_up) # clean dataset
df_eng['nlp_job_description'] = df_eng['nlp_job_description'].progress_apply(myNLP.stem_and_lemmatize_german) # lemmatize strings
df_eng['nlp_job_description'] = df_eng['nlp_job_description'].progress_apply(myNLP.tokenize) # tokenize
df_eng['nlp_job_description'] = df_eng['nlp_job_description'].progress_apply(myNLP.remove_stopwords_german) #remove stopwords from
df_eng['nlp_job_description'] = df_eng['nlp_job_description'].progress_apply(myNLP.re_blob) # reblob
df_eng['nlp_job_description'] = df_eng['nlp_job_description'].progress_apply(lambda x: x.lower().strip())

In [ ]:
display(df_eng['nlp_job_description'].head(10))

In [ ]:
myNLP.word_cloud(df_eng, 'nlp_job_description')

### clustering using BERT Model and KMEAN 

In [ ]:
from transformers import BertModel, BertTokenizer
import torch
from sklearn.cluster import KMeans 
import pandas as pd
import numpy as np


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

In [ ]:
try:
    with open("data/model_data/bert_embedding.pkl", "wb") as file:
        pickle.dump(embeddings, file)
except Exception as e:
    print(f"Error: {e}")

In [ ]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
try:
    with open("data/model_data/bert_embedding.pkl", "rb") as file:
        embeddings = pickle.load(file)
except Exception as e:
    print(f"Error: {e}")

In [ ]:
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
# Initialize variables
silhouette_scores = []
inertia = []
# Loop through different numbers of clusters
for i in range(2, 35):  # Start from 2 as silhouette_score needs at least 2 clusters
    kmeans = KMeans(n_clusters=i, init='k-means++')
    kmeans.fit(embeddings)
    cluster_labels = kmeans.labels_
    
    inertia.append(kmeans.inertia_)
    
    # Calculate the silhouette score and append to the list
    silhouette_avg = silhouette_score(embeddings, cluster_labels)
    silhouette_scores.append(silhouette_avg)
    # print(f"For n_clusters = {i}, the silhouette score is {silhouette_avg}")


In [ ]:
# Plotting the silhouette scores
plt.plot(range(2, 35), silhouette_scores)
plt.title('Silhouette Method')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.show()



plt.plot(range(10, 35), inertia)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()